In [ ]:
import numpy as np
# import the models for further classification experiments
from tensorflow.keras.applications import (vgg16, resnet50, mobilenet, inception_v3)

vgg_model = vgg16.VGG16(weights='imagenet')
inception_model = inception_v3.InceptionV3(weights='imagenet')
resnet_model = resnet50.ResNet50(weights='imagenet')
mobilenet_model = mobilenet.MobileNet(weights='imagenet')

17227776/17225924 [==============================] - 0s 0us/step


In [ ]:
# Import libraries
from __future__ import print_function
import matplotlib
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn as nn
from torch.nn.functional import sigmoid, relu
from scipy.cluster.hierarchy import dendrogram, linkage
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.applications.imagenet_utils import decode_predictions

In [ ]:
!pip install git+git://github.com/jaredwinick/img2vec-keras.git

  Cloning git://github.com/jaredwinick/img2vec-keras.git to /tmp/pip-req-build-vxwkzrl3
  Running command git clone -q git://github.com/jaredwinick/img2vec-keras.git /tmp/pip-req-build-vxwkzrl3
  Created wheel for img2vec-keras: filename=img2vec_keras-0.2-cp37-none-any.whl size=6205 sha256=6c24f61f80005d8295b36d33b92077d017c01b318e8139cf8f1870d5dfc439ca
  Stored in directory: /tmp/pip-ephem-wheel-cache-qj25zo0h/wheels/2e/90/3e/18022740db56735624f1fa778b670e7c013bbf4e9b1802e764
Successfully built img2vec-keras


### A try-out case for generating vector from 'bird.png'

In [ ]:
from img2vec_keras import Img2Vec
img2vec = Img2Vec()
path = "/content/drive/My Drive/1016 Cognition Project/"
x = img2vec.get_vec(path+'images/bird.jpg')

In [ ]:
x

array([0.21992452, 0.05137749, 0.07441771, ..., 0.8241051 , 0.4698608 ,
       0.38602597], dtype=float32)

### Process animal names for matching

In [ ]:
with open(path+'/sem_items_animal_only.txt','r') as fid:
    names_items = np.array([l.strip() for l in fid.readlines()])

In [ ]:
names_items_processed = [name.lower() for name in names_items]
names_items_processed = [name.replace(' ','_') for name in names_items_processed]

In [ ]:
#names_items_processed

### Generate raw vector, as same order as data input

In [ ]:
D = np.loadtxt(path+'sem_data_animal_only.txt')
nobj = len(names_items)

input_pats = D[:,:nobj]
input_pats = torch.tensor(input_pats,dtype=torch.float)
input_v = input_pats[0,:].numpy().astype('bool')
print('Example input pattern:')
print(input_v.astype('int'))

Example input pattern:
[1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0]


In [ ]:
list(input_v.astype('int')).index(1)

0

In [ ]:
def get_vec(input_pat_vec):
  # Binary input vector pattern, same as "Example input pattern" above
  index = input_pat_vec.index(1)
  item = names_items_processed[index] # name of the animal
  image_path = path+'images/'+item+'.jpg'
  image_vec = img2vec.get_vec(image_path)
  return image_vec

In [ ]:
vector_list = []

for item_ind in range(len(input_pats)):
  input_v = input_pats[item_ind,:].numpy().astype('bool')
  input_pat_vec = list(input_v.astype('int'))
  image_vec = get_vec(input_pat_vec)
  vector_list.append(image_vec)

### Get the vectors for 6 general animal group

In [ ]:
category_names = ['amphibians', 'birds', 'fish', 'invertebrates', 'mammals', 'reptiles']
sim_results = {}

In [ ]:
cat_vectors = {}
for cat_name in category_names:
  temp = []
  temp.append(img2vec.get_vec(path+'images/'+cat_name+'1.jpg'))
  temp.append(img2vec.get_vec(path+'images/'+cat_name+'2.jpg'))
  temp.append(img2vec.get_vec(path+'images/'+cat_name+'3.jpg'))
  temp.append(img2vec.get_vec(path+'images/'+cat_name+'4.jpg'))
  cat_vectors[cat_name] = temp

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

def get_category_similarity(item_index, cat_name):
  cat_vec1 = cat_vectors[cat_name][0]
  cat_vec2 = cat_vectors[cat_name][1]
  cat_vec3 = cat_vectors[cat_name][2]
  cat_vec4 = cat_vectors[cat_name][3]
  item_vec = vector_list[item_index]
  X = np.stack([item_vec, cat_vec1, cat_vec2, cat_vec3, cat_vec4])
  Y = X
  sim_matrix = cosine_similarity(X,Y)
  sim_score = sim_matrix[0][1:].mean()
  return sim_score

In [ ]:
for cat_i in category_names:
  temp_storage = []
  for item_ind in range(len(vector_list)):
    item_vec = vector_list[item_ind]
    temp_storage.append(get_category_similarity(item_ind, cat_i))
  sim_results[cat_i] = temp_storage

In [ ]:
#sim_results['birds']